In [1]:
from tinc import *
tclient = TincClient()

Connected to localhost:34450. Server version 1 revision 0
Set working path to C:\Users\Andres\source\repos\casm_viewer\bin\
Validating json with C:\Users\Andres\anaconda3\lib\site-packages\tinc\tinc_cache_schema.json
parsing: C:/Users/Andres/source/repos/vdv_data/MonteCarlo_0/cached_output/graph.png


In [ ]:
tclient._working_path

In [ ]:
import pkg_resources  # part of setuptools|
pkg_resources.require("tinc")[0].version

In [ ]:
# Run this cell if you need verbose debug output
tclient.debug = True

# Parameters and Parameter Spaces

In [ ]:
[p.id for p in tclient.parameters]

In [ ]:
tclient.get_parameter("param_chem_pot(a)").value

In [ ]:
tclient.get_parameter("param_chem_pot(a)").values

In [ ]:
tclient.get_parameter("param_chem_pot(a)").ids

In [ ]:
[ps.id for ps in tclient.parameter_spaces]

In [ ]:
tclient.parameter_spaces[0].get_current_relative_path()

In [ ]:
tclient.parameter_spaces[0].get_root_path()

# Datapools

Through TINC you can access the data from the results file through the data pool interface.

In [ ]:
[dp.id  for dp in tclient.datapools]

In [2]:
results_pool = tclient.get_datapool("resultsData")

In [ ]:
results_pool.slice_cache_dir

In [ ]:
results_pool.get_current_files()

In [ ]:
import json
with open(results_pool.get_current_files()[0]) as f:
    j = json.load(f)

In [ ]:
j.keys()

A data pool allows slicing the dataset in an arbitrary dimension

In [ ]:
slice = results_pool.get_slice("<site_frac(Nb)>", "T")
slice_2 = results_pool.get_slice("<site_frac(O)>", "T")

In [ ]:
%pylab inline

In [ ]:
plot(slice)
plot(slice_2)

In [ ]:
beta = results_pool.get_slice("Beta", "T")

In [ ]:
beta

# Parameter Callbacks

Create a parameter through the TINC client. All parameters assigned to the "casm" group will be displayed in the CASM viewer interface.

In [ ]:
p = tclient.create_parameter(Parameter,\
                                    "myparam","casm",-0.375,0.375,\
                                    [-0.375,-0.125,0.125,0.375],0.125)

In [ ]:
p.value

In [ ]:
p.value = 0.125

In [ ]:
def func(value):
    print(value)    

Register the func as a callback to the 'p' parameter. This function will be called whenever the parameter value changes, whether from the C++ GUI or from this notebook.

In [ ]:
p.register_callback(func)

In [ ]:
p.value = 0.1

In [ ]:
q = tclient.create_parameter(Parameter,\
                                 "output","casm",-10,10)

In [ ]:
def func(value):
    print(value)
    q.value = value * 3 +1 

Even though it has the same name, we must register the function.

In [ ]:
p.register_callback(func)

```Trigger``` parameters are displayed as buttons in the GUI.

In [ ]:
t = tclient.create_parameter(Trigger,\
                                    "compute","casm")

In [ ]:
def compute(value):
    print("hello")

In [ ]:
t.register_callback(compute)

```ParameterString``` parameters are used to display text.

In [ ]:
s = tclient.create_parameter(ParameterString,\
                                    "text","casm")

In [ ]:
s.value

In [ ]:
s.value = "hello CASM"

# Generating a graph

In [ ]:
[db.id for db in tclient.disk_buffers]

In [ ]:
g3 = tclient.get_disk_buffer("graph3")

In [ ]:
def graph(value):
    x=np.linspace(0,1,10);
    y=np.random.random(10);
    plt.plot(x,y);
    plt.title(os.getcwd().split("/")[-1].replace("_",":"));
    name = g3.get_filename_for_writing()
    plt.savefig(name);
    g3.done_writing_file(name)

Register the ```graph()``` function to the "compute" trigger. Whenever the button is clicked, a new graph will be generated

In [ ]:
t.register_callback(graph)

In [ ]:
g2 = tclient.get_disk_buffer("graph2")

def update_plot(value):
    print(" ******** Update before get_slice")
    slice = results_pool.get_slice("<site_frac(Nb)>", "T")
    plt.clf()
    plt.plot(slice)
    plt.title(str(value))
    name = g2.get_filename_for_writing()
    plt.savefig(name);
    g2.done_writing_file(name)
    print(value)

In [ ]:
pwd

In [ ]:
update_plot(12)

When registering a callback that calls certain functions, you will get a warning:

In [ ]:
t.register_callback(update_plot)

In [ ]:
t.remove_callback(update_plot)

Register the callback to run asynchronously to avoid this problem:

In [ ]:
t.register_callback_async(update_plot)

# Atom buffers

In [3]:
buf = tclient.get_disk_buffer("atomRender_buffer")

The data by default is a flat list of x,y,z and hue

In [4]:
buf.data

array([  3.8739953,  -1.0790416,   2.5178242, ..., -42.083927 ,
        46.040554 ,   0.8333333], dtype=float32)

In [5]:
buf.data = [0,0,0,0.5, 10, 10, 10, 0.9]

sent disk buffer filename: positions_1.nc
parsing: C:/Users/Andres/source/repos/vdv_data/MonteCarlo_0/cached_output/graph.png


# Caching

In [ ]:
def process(T):
    print(T)

In [ ]:
ps = tclient.get_parameter_space("casmParameters")

In [ ]:
ps.run_process(process)

In [ ]:
ps.get_parameter("T").value = 300

In [ ]:
ps.run_process(process)

In [ ]:
ps.enable_cache()

In [ ]:
ps.run_process(process)

In [ ]:
ps.run_process(process)

In [ ]:
ps.sweep(process)